# Linux Kernel Hello World

This is simple kernel module.  
- Print "Hello, World!" during module installation.
- Print "Good Bye, World!" when the module is uninstalled.

## 1. The C source file

### 1.1 License

The `MODULE_LICENSE()` macro specifies the copyright license for this file. Loading a non-GPL module into memory results in the tainted flag being set in the kernel.

In [1]:
%%writefile hello-world.c
#include <linux/init.h>
#include <linux/module.h>
#include <linux/kernel.h>

MODULE_LICENSE("GPL");
MODULE_AUTHOR("Clymber Loong");
MODULE_DESCRIPTION("A Hello, World Module");

Overwriting hello-world.c


### 1.2 Define module init and exit functions

The init and exit functions must have the form as:
```c
int modulename_init(void);
void modulename_exit(void);
```
If initialization was successful, the function must return zero. On failure, the function must unwind any initialization and return nonzero.   
If this file were compiled into the static kernel image, the exit function would not be included, and it would never be invoked because if it were not a module, the code could never be removed from memory.

In [2]:
%%writefile -a hello-world.c

static int hello_init(void)
{
    printk(KERN_ALERT "Hello, World!\n");
    return 0;
}

static void hello_exit(void)
{
    printk(KERN_ALERT "Bye Bye, World!\n");
}

Appending to hello-world.c


### 1.3 Register module's entry point and exit point

In [3]:
%%writefile -a hello-world.c

module_init(hello_init);
module_exit(hello_exit);

Appending to hello-world.c


The `hello_init()` function is registered as this module’s entry point via `module_init()`.The kernel invokes `hello_init()`
when the module is loaded.  
The `module_exit()` function registers a module’s exit point. In this example, we register the function `hello_exit()`.

## 2. The Makefile

In [4]:
%%writefile Makefile
obj-m = hello-world.o

KERN_BUILD = /lib/modules/$(shell uname -r)/build
MODULE_DIR = $(PWD)

all:
	make -C $(KERN_BUILD) M=$(MODULE_DIR) modules

clean:
	make -C $(KERN_BUILD) M=$(MODULE_DIR) clean


Overwriting Makefile


The `KERN_BUILD` defines the path to the linux kernel source tree within which we want to build the module, and it should be built before hand.  
The `MODULE_DIR` defines the path to the module that we want to build.  
The `make -C $(KERN_BUILD)` shows that `make` command will change its CWD to `$(KERN_BUILD)` before build. It means that the module will actually be built within the linux kernel source tree.

## 3. Build the module

In [5]:
! make

make -C /lib/modules/6.1.28/build M=/home/clymber/Studio/hello-kernel modules
make[1]: Entering directory '/home/clymber/Studio/@linux-6.1.28'
  CC [M]  /home/clymber/Studio/hello-kernel/hello-world.o
  MODPOST /home/clymber/Studio/hello-kernel/Module.symvers
  LD [M]  /home/clymber/Studio/hello-kernel/hello-world.ko
make[1]: Leaving directory '/home/clymber/Studio/@linux-6.1.28'


In [6]:
! ls -lh *.ko

-rw-rw-r-- 1 clymber clymber 105K May 13 00:34 hello-world.ko


## 4. Load the module into kernel

The simplest way to load a module is via insmod. The insmod program does not perform any dependency resolution or advanced error checking.

In [7]:
! sudo insmod hello-world.ko

In [8]:
! sudo dmesg | grep -F 'Hello, World!'

[ 5631.922223] Hello, World!


## 5. Unload the module from kernel

In [9]:
! sudo rmmod hello-world.ko

In [10]:
! sudo dmesg | grep -F 'Bye Bye, World!'

[ 5632.185099] Bye Bye, World!


## 6. Clean project

In [11]:
! make clean

make -C /lib/modules/6.1.28/build M=/home/clymber/Studio/hello-kernel clean
make[1]: Entering directory '/home/clymber/Studio/@linux-6.1.28'
  CLEAN   /home/clymber/Studio/hello-kernel/Module.symvers
make[1]: Leaving directory '/home/clymber/Studio/@linux-6.1.28'
